In [2]:
import json
import pandas as pd

def safe_json_load(value):
    if not value or pd.isna(value) or (isinstance(value, str) and value.strip() == ""):
        return {}
    try:
        if isinstance(value, str):
            return json.loads(value)
        return value
    except json.JSONDecodeError:
        return {}

def analyze_json_references(data):
    """
    Process a JSON object (or dict) to count and analyze properties
    with 'referenced' and 'text_reference'. Returns a summary dictionary.
    """
    true_count = 0
    false_count = 0
    false_details = {}  # Initialize as dict instead of list
  
    def process_json(obj, parent_key=None):
        nonlocal true_count, false_count, false_details
        if isinstance(obj, dict):
            # Check if current dict has both "referenced" and "text_reference"
            if "referenced" in obj and "text_reference" in obj:
                if obj["referenced"] is True:
                    true_count += 1
                else:
                    false_count += 1
                    # Use parent key if available; otherwise, 'unknown'
                    prop_name = parent_key if parent_key is not None else 'unknown'
                    false_details[prop_name] = obj["text_reference"]
            # Recursively process each key-value pair
            for key, value in obj.items():
                process_json(value, parent_key=key)
        elif isinstance(obj, list):
            for item in obj:
                process_json(item, parent_key=parent_key)

    process_json(data)
    total_references = true_count + false_count
    percentage_true = (true_count / total_references * 100) if total_references else 0
   
    return {
        "prompt_related_score": percentage_true,
        "total_references": total_references,
        "true_references": true_count,
        "false_references": false_count,
        "false_references_details": false_details
    }

def analyze_json_column(json_value):
    """
    Wrapper function to process a DataFrame cell.
    Expects json_value to be either a JSON string or a dict.
    """
    if isinstance(json_value, str):
        data = json.loads(json_value)
    else:
        data = json_value
    return analyze_json_references(data)


def extract_all_schema_keys(schema):
    keys = set()
    def traverse(obj, parent=""):
        if isinstance(obj, dict):
            # Traverse keys under "properties"
            if "properties" in obj:
                for key, value in obj["properties"].items():
                    # You can either capture the full path or just the key
                    # Here, we capture just the key name
                    keys.add(key)
                    traverse(value, key)
            # Also traverse "additionalProperties" if present
            if "additionalProperties" in obj and isinstance(obj["additionalProperties"], dict):
                traverse(obj["additionalProperties"], parent)
            # Traverse oneOf, anyOf, allOf arrays if present
            for combiner in ["oneOf", "anyOf", "allOf"]:
                if combiner in obj and isinstance(obj[combiner], list):
                    for item in obj[combiner]:
                        traverse(item, parent)
            # Traverse "items" (for arrays)
            if "items" in obj:
                traverse(obj["items"], parent)
        elif isinstance(obj, list):
            for item in obj:
                traverse(item, parent)
    traverse(schema)
    return keys


def extract_referenced_json_keys_nonmetadata(data):
    """
    Recursively extracts all keys from a JSON object as hierarchical dot paths,
    but ignores common metadata keys like "type", "referenced", "text_reference",
    and additional ones such as "minimum", "maximum", "pattern", "enum", "format", and "items".
    """
    metadata_keys = {"type", "referenced", "text_reference", "minimum", "maximum", "pattern", "enum", "format", "items"}
    keys = set()
    
    def traverse(obj, parent=""):
        if isinstance(obj, dict):
            for key, value in obj.items():
                # Skip metadata keys
                if key in metadata_keys:
                    continue
                current_path = f"{parent}.{key}" if parent else key
                keys.add(current_path)
                traverse(value, current_path)
        elif isinstance(obj, list):
            for item in obj:
                traverse(item, parent)
    
    traverse(data)
    return keys


def analyze_json_and_schema(referenced_json, schema_json):
    """
    Combines analysis of the referenced JSON with schema matching.
    Returns the analysis dict with the following additional keys:
      - schema_keys_count: number of keys (without parent paths) in the schema.
      - referenced_json_key_count: number of keys (without parent paths) in the referenced JSON.
      - matching_properties_count: number of keys that appear in both.
      - non_matching_properties_count: number of keys in the schema that are not present in the referenced JSON.
      - non_matching_properties: sorted list of keys (from the schema) that are missing in the referenced JSON.
    
    The original analysis from analyze_json_column(referenced_json) is preserved.
    """
    # Get the original analysis from the referenced JSON.
    analysis = analyze_json_column(referenced_json)
    
    # Use your helper functions that extract keys without parent paths.
    schema_keys_str = get_sorted_schema_keys_without_parents(schema_json)
    referenced_keys_str = get_referenced_json_keys_without_parents(referenced_json)
    
    # Convert the comma-separated strings into sets of keys.
    schema_keys_set = set([k.strip() for k in schema_keys_str.split(",") if k.strip()])
    referenced_keys_set = set([k.strip() for k in referenced_keys_str.split(",") if k.strip()])
    
    # Calculate key counts and matching statistics.
    schema_keys_count = len(schema_keys_set)
    referenced_json_key_count = len(referenced_keys_set)
    matching_properties_count = len(schema_keys_set.intersection(referenced_keys_set))
    
    # Here we list only the keys defined in the schema that are missing in the referenced JSON.
    non_matching = schema_keys_set - referenced_keys_set
    non_matching_properties_count = len(non_matching)
    
    # Append the matching stats to the analysis.
    analysis["schema_keys_count"] = schema_keys_count
    analysis["referenced_json_key_count"] = referenced_json_key_count
    analysis["matching_properties_count"] = matching_properties_count
    analysis["non_matching_properties_count"] = non_matching_properties_count
    analysis["non_matching_properties"] = sorted(list(non_matching))
    
    return analysis

# Helper function to get sorted schema keys as a comma-separated string
def get_sorted_schema_keys(schema_value):
    schema_data = safe_json_load(schema_value)
    keys = extract_all_schema_keys(schema_data)
    return ", ".join(sorted(keys))

# Helper function to get sorted data keys as a comma-separated string
def get_referenced_json_keys(data_value):
    data = safe_json_load(data_value)
    keys = extract_referenced_json_keys_nonmetadata(data)
    return ", ".join(sorted(keys))

def get_sorted_schema_keys_without_parents(schema_value):
    # Call the original function to get the full hierarchical keys as a comma-separated string
    full_keys_str = get_sorted_schema_keys(schema_value)
    # Split the string into a list and take only the last segment of each key
    key_list = [k.strip().split('.')[-1] for k in full_keys_str.split(",") if k.strip()]
    # Optionally, remove duplicates and sort the keys
    unique_keys = sorted(set(key_list))
    return ", ".join(unique_keys)

def get_referenced_json_keys_without_parents(data_value):
    full_keys_str = get_referenced_json_keys(data_value)
    key_list = [k.strip().split('.')[-1] for k in full_keys_str.split(",") if k.strip()]
    unique_keys = sorted(set(key_list))
    return ", ".join(unique_keys)

def get_schema_key_count(schema_value):
    # Get the comma-separated string of keys without parent paths.
    keys_str = get_sorted_schema_keys_without_parents(schema_value)
    # Split into a list and strip any extra whitespace, then create a set to deduplicate.
    keys_set = set(k.strip() for k in keys_str.split(",") if k.strip())
    return len(keys_set)

def get_referenced_json_key_count(data_value):
    # Get the comma-separated string of keys without parent paths.
    keys_str = get_referenced_json_keys_without_parents(data_value)
    # Split into a list, strip whitespace, and deduplicate.
    keys_set = set(k.strip() for k in keys_str.split(",") if k.strip())
    return len(keys_set)

def create_key_comparison_aligned(row):
    # Get lists of keys from each column (assuming they're comma‑separated strings)
    schema_keys = {k.strip() for k in row["schema_keys"].split(",") if k.strip()}
    data_keys   = {k.strip() for k in row["referenced_json_keys"].split(",") if k.strip()}
    # Create a sorted list of all keys from both sets
    all_keys = sorted(schema_keys.union(data_keys))
    
    rows = []
    for key in all_keys:
        rows.append({
            "Key": key,
            "Schema Key": key if key in schema_keys else "",
            "Referenced JSON Key": key if key in data_keys else ""
        })
    return pd.DataFrame(rows)


In [5]:
import pandas as pd
import json

# load and concat all csv files 01_output_0_100, 01_output_100_200, 01_output_100_200, 01_output_200_300, 01_output_300_400 with a for loop

all_dfs = []
base_path = '/Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/'
file_ranges = [(0, 100), (100, 200), (200, 300), (300, 400)]

for start, end in file_ranges:
    file_path = f"{base_path}01_output_{start}_{end}.csv"
    try:
        temp_df = pd.read_csv(file_path)
        all_dfs.append(temp_df)
        print(f"Loaded {file_path}, shape: {temp_df.shape}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Concatenate all dataframes
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataframe shape: {df.shape}")
else:
    print("No dataframes to concatenate")


def safe_analyze(row):
    json_str = row['REFERENCED_JSON_FORMATED']
    # Debug: Print problematic rows
    if not json_str or not json_str.strip():
        print(f"Row {row.name} has an empty or whitespace JSON string.")
        return {}
    try:
        return analyze_json_and_schema(json_str, row['schema'])
    except json.JSONDecodeError as e:
        print(f"Error processing row {row.name}: {e}")
        return {}  # Return a default value when JSON decoding fails

df['summary'] = df.apply(lambda row: json.dumps(safe_analyze(row), indent=2), axis=1)


#add a column flag = true if  "prompt_related_score": 0," in summary column
df['flag'] = df['summary'].str.contains('"prompt_related_score": 0,')

df['flag'].value_counts()



Loaded /Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_0_100.csv, shape: (100, 13)
Loaded /Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_100_200.csv, shape: (100, 13)
Error loading /Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_200_300.csv: [Errno 2] No such file or directory: '/Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_200_300.csv'
Error loading /Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_300_400.csv: [Errno 2] No such file or directory: '/Users/dan.rambado/Documents/reviewing_json_schema_dataset/output/existing_981_tasks/01_output_300_400.csv'
Combined dataframe shape: (200, 13)
Error processing row 56: Invalid \escape: line 149 column 252 (char 6623)
Error processing row 67: Expecting value: line 9 column 23 (char 325)
Error processing row 1

flag
False    195
True       5
Name: count, dtype: int64

In [6]:
df.columns

Index(['languageCode', 'INTERNAL_ID', 'CUSTOMER_ID', 'PARSED_CUST_ID',
       'TASK_ID', 'BATCH_ID', 'prompt', 'schema', 'response',
       'MR_EVAL_SUB_SCHEMA', 'REFERENCED_JSON', 'SCHEMA_SIMPLIFIED',
       'REFERENCED_JSON_FORMATED', 'summary', 'flag'],
      dtype='object')

In [8]:
df.rename(columns={'REFERENCED_JSON_FORMATED': 'reference_JSON',
                   'INTERNAL_ID':'internal_id',
                   'TASK_ID': 'TASK'}, inplace=True)

# keep the columns needed
df = df[['TASK','languageCode', 'internal_id', 'prompt', 'schema', 'reference_JSON', 'summary']]


# remove rows that contains "prompt_related_score": 0," in summary column
df = df[~df['summary'].str.contains('"prompt_related_score": 0,')]

# replace all the value in 'languageCode' for "en_US"
df['languageCode'] = 'en_US'

# save to csv
df.to_csv(f'{base_path}v2_batch_005_mar5.csv', index=False)
